In [1]:
from simModel.common.dataQueue import RenderQueue, FocusPos, SimPause
from simModel.common.dataBase import Database
from trafficManager.vehicle import Vehicle
import csv
import numpy as np
import object_list_with_TL_pb2

In [38]:
def scalr_and_heading_to_vector(scalar, heading):
    return scalar * np.array([np.cos(heading), np.sin(heading)])

def assign_Object(id,x,y,velQ,hdg,accQ,scf,length,width):
    # create ego object and add that to timeslot
    ftlx_logiq_object = object_list_with_TL_pb2.Object()
    ftlx_logiq_object.tracking_id = str(id)
    ftlx_logiq_object.kind = object_list_with_TL_pb2.ObjectKind.Value('KIND_CAR')
    ftlx_logiq_object.position.x = float(x)
    ftlx_logiq_object.position.y = float(y)
    ftlx_logiq_object.position.z = float(0)
    ftlx_logiq_object.velocity.x = scalr_and_heading_to_vector(velQ[0], hdg)[0]
    ftlx_logiq_object.velocity.y = scalr_and_heading_to_vector(velQ[0], hdg)[1]
    ftlx_logiq_object.acceleration.x = scalr_and_heading_to_vector(accQ[0], hdg)[0]
    ftlx_logiq_object.acceleration.y = scalr_and_heading_to_vector(accQ[0], hdg)[1]
    ftlx_logiq_object.yaw = float(hdg)
    ftlx_logiq_object.lane = int(0)
    ftlx_logiq_object.position_in_lane = float(scf)
    ftlx_logiq_object.length = float(length)
    ftlx_logiq_object.width = float(width)
    ftlx_logiq_object.height = float(2)
    return ftlx_logiq_object

db_file_name = "simulationDatabase1"

db = Database(db_file_name + ".db")

total_time_step = db.getRunTime()

egoID = -1

ego = Vehicle(egoID)
vehInAoI = {}
outOfAoI = {}

def create_LLA(latitude,longitude,altitude):
    # WGS84 LLA format (e.g. GPS) 
    lla = object_list_with_TL_pb2.GlobalPosition()
    lla.latitude = latitude
    lla.longitude = longitude
    lla.altitude = altitude
    return lla

stepTime = 0.25 # in seconds

# create root msg
root = object_list_with_TL_pb2.Root()
root.is_absolute = True # True: absolute coordinates, False: relative coordinates
root.step_time = int(stepTime*1000) # in milliseconds
root.start_time = 0.0 # in milliseconds
# Used to position the local frame origin and rotation with respect to the global frame
local_frame = object_list_with_TL_pb2.LocalFrameOriginPosition() 
local_frame.lla.CopyFrom(create_LLA(0.0, 0.0, 0.0)) 
local_frame.yaw = 0.0
root.version = int(1)
root.origin_start_time = 0.0 #The original-global start time (before zeroing it out)

for timeStep in range(1, total_time_step+1):
    vehInfo, tlsInfo = db.getDB(timeStep)

    # create a timeslot msg
    timeslot = root.times.add()
    # timeslot = object_list_with_TL_pb2.TimeSlot()
    # set the time of the timeslot
    timeslot.time = int(timeStep*stepTime*1000)
    
    runFinlished = False
    vehRunning = {}
    for vehId in vehInfo:
        vehRunning[vehId] = vehInfo[vehId]
    if vehRunning:
        # 如果 self.ego.id 等于 -1：这表示当前没有设置 ego 车辆。我们将 self.vehRunning 中的第一个车辆设置为 ego 车辆
        if ego.id == -1:
            ego = list(vehRunning.values())[0]
        # 如果 self.ego.id 大于 0 且存在于 self.vehRunning 中：这表示 ego 车辆仍在运行。我们将 self.ego 更新为 self.vehRunning 中对应的车辆。
        elif ego.id > 0 and ego.id in vehRunning:
            ego = vehRunning[ego.id]
        else:
            runFinlished = True
    else:
        runFinlished = True
    
    if not runFinlished:
        egoObj = assign_Object(ego.id, ego.x, ego.y, ego.velQ, ego.hdg, ego.accQ, ego.scf, ego.length, ego.width)
        timeslot.ego.CopyFrom(egoObj)

        # get other cars by Euclidean distance
        for vehId in vehRunning:
            veh = vehRunning[vehId]
            if ego.id != veh.id:
                # if the distance between ego and veh is less than the ego's detection area, then veh is in the AoI of ego
                if np.hypot(ego.scf - veh.scf, ego.tcf - veh.tcf) < ego.deArea:
                    vehInAoI[vehId] = veh
                else:
                    outOfAoI[vehId] = veh
        if vehInAoI:
            for vehId in vehInAoI:
                veh = vehInAoI[vehId]
                vehObj = assign_Object(veh.id, veh.x, veh.y, veh.velQ, veh.hdg, veh.accQ, veh.scf, veh.length, veh.width)
                timeslot.objects.append(vehObj)

# save root to pb file
with open(db_file_name + ".pb", 'wb') as f:
    f.write(root.SerializeToString())
print("pb file " + db_file_name + ".pb" + " has been created.")


pb file simulationDatabase1.pb has been created.
